In [27]:
import pandas as pd 
import numpy as np



In [28]:

my_file = '3_and_4_Loan_Data.csv'
df = pd.read_csv(my_file)

fico_scores = df['fico_score'].tolist()
defaults = df['default'].tolist()

print(fico_scores)

print(defaults)

[605, 572, 602, 612, 631, 697, 722, 545, 676, 447, 640, 636, 527, 638, 616, 644, 606, 727, 627, 581, 667, 480, 677, 726, 596, 605, 636, 642, 596, 667, 580, 697, 542, 668, 787, 673, 609, 541, 560, 607, 631, 746, 530, 615, 552, 606, 599, 679, 603, 764, 736, 751, 610, 663, 737, 629, 689, 587, 692, 706, 515, 702, 638, 612, 583, 720, 607, 750, 662, 563, 611, 754, 663, 709, 576, 611, 669, 612, 720, 589, 578, 652, 687, 633, 569, 657, 754, 642, 708, 627, 745, 685, 679, 697, 603, 555, 687, 625, 581, 633, 622, 610, 584, 628, 724, 620, 563, 678, 602, 672, 724, 626, 749, 582, 693, 632, 655, 693, 721, 710, 541, 587, 581, 572, 706, 582, 710, 705, 672, 568, 511, 640, 709, 707, 651, 662, 766, 692, 590, 657, 592, 718, 720, 636, 654, 513, 587, 671, 655, 609, 641, 621, 658, 594, 622, 636, 603, 619, 646, 533, 672, 598, 726, 641, 669, 556, 659, 735, 571, 509, 626, 678, 634, 657, 637, 684, 692, 723, 562, 713, 598, 584, 651, 626, 583, 674, 633, 559, 684, 735, 636, 756, 653, 685, 682, 774, 697, 614, 584, 641,

In [29]:
def log_likelihood(start, end, cum_defaults):
    k = cum_defaults[end] - cum_defaults[start]
    n = end - start
    
    if k == 0 or k==n:
        return 0
    p = k/n
    
    return k * np.log(p) + (n-k) * np.log(1-p)
    

In [30]:
def optimal_bucketing(fico_scores, default_labels, B):
    N = len(fico_scores)
    data = sorted(zip(fico_scores, default_labels), key=lambda x: x[0])
  
    scores = [x[0] for x in data]
    labels = [x[1] for x in data]
    
    cum_defaults = [0] * (N+1)
    for i in range(N):
        cum_defaults[i+1] = cum_defaults[i] + labels[i]
        
        
    dp = np.full((N+1,B+1), -np.inf)
    backtrack = np.full((N+1, B+1), -1)
    dp[0][0] = 0.0
    
    for b in range(1, B+1):
        for i in range(b, N+1):
            for j in range(b-1, i):
                ll = log_likelihood(j, i, cum_defaults)
                candidate = dp[j, b-1] + ll
                if candidate > dp[i][b]:
                    dp[i][b] = candidate
                    backtrack[i][b] = j 
                    
    
    buckets = []
    i,b = N, B
    while b > 0:
        j = backtrack[i,b]
        buckets.append(scores[j])
        i, b = j, b-1
        
    return sorted(buckets)



In [ ]:
data = sorted(zip(fico_scores, defaults), key=lambda x: x[0])
  
print(data)

    
scores = [x[0] for x in data]
print(scores)
labels = [x[1] for x in data]
print(labels)

[(408, 0), (409, 1), (418, 1), (425, 1), (438, 1), (440, 1), (441, 1), (444, 0), (447, 0), (449, 1), (450, 0), (450, 0), (455, 1), (455, 1), (456, 1), (456, 1), (456, 1), (456, 1), (459, 0), (459, 0), (460, 0), (461, 1), (462, 1), (462, 1), (462, 0), (462, 1), (463, 1), (464, 1), (465, 1), (465, 1), (466, 1), (466, 1), (467, 1), (467, 0), (468, 1), (468, 1), (468, 1), (469, 1), (469, 1), (470, 1), (471, 0), (471, 1), (472, 1), (473, 1), (474, 1), (474, 1), (475, 0), (475, 1), (476, 1), (476, 0), (477, 1), (479, 1), (479, 1), (479, 0), (480, 1), (480, 1), (480, 0), (481, 0), (481, 1), (481, 1), (481, 0), (481, 0), (481, 1), (481, 0), (481, 1), (482, 1), (482, 0), (482, 1), (483, 0), (483, 1), (483, 1), (483, 1), (483, 1), (483, 0), (484, 0), (484, 1), (485, 1), (485, 1), (485, 1), (485, 1), (486, 1), (486, 1), (486, 0), (487, 1), (487, 1), (487, 0), (487, 1), (487, 1), (488, 1), (488, 0), (488, 1), (489, 1), (489, 0), (489, 1), (490, 1), (490, 1), (490, 1), (490, 1), (490, 1), (490, 1),

In [32]:
boundaries = optimal_bucketing(fico_scores, defaults, B=5)
print("Bucket boundaries:", boundaries)


Bucket boundaries: [408, 553, 611, 649, 717]
